In [10]:
import requests
import ast
import pandas as pd


# URL da API
url = "https://api.openligadb.de/getmatchdata/wwc/2023"

# Fazendo a requisição para a API
response = requests.get(url)

# Verificando se a requisição foi bem-sucedida
if response.status_code == 200:
    data = response.json()
    print("Dados extraídos com sucesso!")
else:
    print(f"Erro na requisição: {response.status_code}")
    data = []

# Convertendo os dados para um DataFrame do Pandas
df = pd.DataFrame(data)

# Exibindo as primeiras linhas e as colunas do DataFrame
print(df.head())
print(df.columns)

# Salvando os dados extraídos em um arquivo CSV para análise posterior
df.to_csv("match_data.csv", index=False)

print("Dados salvos em match_data.csv")


Dados extraídos com sucesso!
   matchID        matchDateTime               timeZoneID  leagueId  \
0    67255  2023-07-20T09:00:00  W. Europe Standard Time      4640   
1    67256  2023-07-20T12:00:00  W. Europe Standard Time      4640   
2    67257  2023-07-21T04:30:00  W. Europe Standard Time      4640   
3    67258  2023-07-21T07:30:00  W. Europe Standard Time      4640   
4    67259  2023-07-21T09:30:00  W. Europe Standard Time      4640   

                    leagueName  leagueSeason leagueShortcut  \
0  FIFA Women´s World Cup 2023          2023            wwc   
1  FIFA Women´s World Cup 2023          2023            wwc   
2  FIFA Women´s World Cup 2023          2023            wwc   
3  FIFA Women´s World Cup 2023          2023            wwc   
4  FIFA Women´s World Cup 2023          2023            wwc   

       matchDateTimeUTC                                              group  \
0  2023-07-20T07:00:00Z  {'groupName': 'Group Matches 1', 'groupOrderID...   
1  2023-07-20T1

In [11]:
# Carregar o arquivo CSV
file_path = 'match_data.csv'
df = pd.read_csv(file_path)

In [12]:
# Função para verificar se 'Group Matches' não está no 'groupName'
# Esta funcao isola apenas as partidas da etapa eliminatoria
def filter_group(group_str):
    try:
        group_dict = ast.literal_eval(group_str)
        return 'Group Matches' not in group_dict['groupName']
    except (ValueError, KeyError):
        return False

In [13]:
# Aplicar a função de filtro no DataFrame
filtered_df = df[df['group'].apply(filter_group)]

# Verificar as primeiras linhas do DataFrame filtrado
print(filtered_df.head())

# Salva uma versao filtrada no path
filtered_df.to_csv("match_data_filtered.csv", index=False)

    matchID        matchDateTime               timeZoneID  leagueId  \
48    67514  2023-08-05T07:00:00  W. Europe Standard Time      4640   
49    67515  2023-08-05T10:00:00  W. Europe Standard Time      4640   
50    67516  2023-08-06T04:00:00  W. Europe Standard Time      4640   
51    67517  2023-08-06T11:00:00  W. Europe Standard Time      4640   
52    67512  2023-08-07T09:30:00  W. Europe Standard Time      4640   

                     leagueName  leagueSeason leagueShortcut  \
48  FIFA Women´s World Cup 2023          2023            wwc   
49  FIFA Women´s World Cup 2023          2023            wwc   
50  FIFA Women´s World Cup 2023          2023            wwc   
51  FIFA Women´s World Cup 2023          2023            wwc   
52  FIFA Women´s World Cup 2023          2023            wwc   

        matchDateTimeUTC                                              group  \
48  2023-08-05T05:00:00Z  {'groupName': 'Round of 16', 'groupOrderID': 4...   
49  2023-08-05T08:00:00Z  {'gr

In [14]:

# Carregar o arquivo CSV 
file_path = 'match_data_filtered.csv'
df = pd.read_csv(file_path)

In [15]:
# Inicializando um dicionário para armazenar os gols por seleção
goals_by_team = {}

# Iterando sobre as partidas
for _, match in df.iterrows():
    try:
        # Converter as strings JSON para dicionários
        team1 = ast.literal_eval(match['team1'])
        team2 = ast.literal_eval(match['team2'])
        match_results = ast.literal_eval(match['matchResults'])
        goals = ast.literal_eval(match['goals'])

        team1_name = team1['teamName']
        team2_name = team2['teamName']

        # Encontrar o resultado final no array de resultados
        final_result = [result for result in match_results if result['resultName'] == 'Endergebnis'][0]
        goals1 = final_result['pointsTeam1']
        goals2 = final_result['pointsTeam2']

        # Subtrair os gols de pênalti
        penalties_result = [result for result in match_results if result['resultName'] == 'nach Elfmeterschießen']
        if penalties_result:
            penalties = penalties_result[0]
            goals1 -= penalties['pointsTeam1']
            goals2 -= penalties['pointsTeam2']

        if team1_name not in goals_by_team:
            goals_by_team[team1_name] = 0
        if team2_name not in goals_by_team:
            goals_by_team[team2_name] = 0

        goals_by_team[team1_name] += goals1
        goals_by_team[team2_name] += goals2

        # Mensagens de depuração
        print(f"Partida processada: {team1_name} vs {team2_name}")
        print(f"Gols nesta partida: {team1_name} - {goals1}, {team2_name} - {goals2}")
        print(f"Total de gols até agora: {team1_name} - {goals_by_team[team1_name]}, {team2_name} - {goals_by_team[team2_name]}")

    except KeyError as e:
        print(f"Erro ao processar a partida: {e}")
    except (ValueError, SyntaxError) as e:
        print(f"Erro ao decodificar dados de dicionário: {e}")

# Convertendo o dicionário para um DataFrame
goals_df = pd.DataFrame(list(goals_by_team.items()), columns=['Team', 'Goals'])

# Salvando os dados manipulados em um arquivo CSV para análise posterior
goals_df.to_csv("goals_by_team.csv", index=False)

print("Dados manipulados e salvos em goals_by_team.csv")


Partida processada: Schweiz Frauen vs Spanien Frauen
Gols nesta partida: Schweiz Frauen - 1, Spanien Frauen - 5
Total de gols até agora: Schweiz Frauen - 1, Spanien Frauen - 5
Partida processada: Japan Frauen vs Norwegen Frauen
Gols nesta partida: Japan Frauen - 3, Norwegen Frauen - 1
Total de gols até agora: Japan Frauen - 3, Norwegen Frauen - 1
Partida processada: Niederlande Frauen vs Südafrika Frauen
Gols nesta partida: Niederlande Frauen - 2, Südafrika Frauen - 0
Total de gols até agora: Niederlande Frauen - 2, Südafrika Frauen - 0
Partida processada: Schweden Frauen vs USA Frauen
Gols nesta partida: Schweden Frauen - 0, USA Frauen - 0
Total de gols até agora: Schweden Frauen - 0, USA Frauen - 0
Partida processada: England Frauen vs Nigeria Frauen
Gols nesta partida: England Frauen - 0, Nigeria Frauen - 0
Total de gols até agora: England Frauen - 0, Nigeria Frauen - 0
Partida processada: Australien Frauen vs Dänemark Frauen
Gols nesta partida: Australien Frauen - 2, Dänemark Fraue